In [1]:
#!/usr/bin/env python3

In [2]:
%matplotlib inline
import numpy as np 
import time 
from datetime import timedelta
import tensorflow as tf
from readImages import *
from build_tensorflow_graph import *
import matplotlib.pyplot as plt

In [3]:
#
## ---------- DCGAN Architecture ----------
#

numFilters1 = 32
numFilters2 = 64
numFilters3 = 128
numFilters4 = 256

# Convolution Layer 1
cnnArchitecture1 = CNN_Architecture(numFilters = numFilters1, 
                                    filterSize = (4, 4), 
                                    strides = 1, 
                                    toPadding = True, 
                                    useReLU = True, 
                                    numInputChannels = 3, 
                                    maxPoolingSize=(2, 2))

dcnnArchitecture1 = CNN_Architecture(numFilters = 3, 
                                    filterSize = (4, 4), 
                                    strides = 1, 
                                    toPadding = True, 
                                    useReLU = False, 
                                    numInputChannels = numFilters1, 
                                    maxPoolingSize=(2, 2))

# Convolution Layer 2
cnnArchitecture2 = CNN_Architecture(numFilters = numFilters2, 
                                    filterSize = (3, 3), 
                                    strides = 1, 
                                    toPadding = True, 
                                    useReLU = True, 
                                    numInputChannels = numFilters1, 
                                    maxPoolingSize=(2, 2))
dcnnArchitecture2 = CNN_Architecture(numFilters = numFilters1, 
                                    filterSize = (3, 3), 
                                    strides = 1, 
                                    toPadding = True, 
                                    useReLU = True, 
                                    numInputChannels = numFilters2, 
                                    maxPoolingSize=(2, 2))

# Convolution Layer 3
cnnArchitecture3 = CNN_Architecture(numFilters = numFilters3,
                                    filterSize = (3, 3), 
                                    strides = 1, 
                                    toPadding = True, 
                                    useReLU = True, 
                                    numInputChannels = numFilters2, 
                                    maxPoolingSize=(2, 2))
dcnnArchitecture3 = CNN_Architecture(numFilters = numFilters2,
                                    filterSize = (3, 3), 
                                    strides = 1, 
                                    toPadding = True, 
                                    useReLU = True, 
                                    numInputChannels = numFilters3, 
                                    maxPoolingSize=(2, 2))

# Convolution Layer 4
cnnArchitecture4 = CNN_Architecture(numFilters = numFilters4,
                                    filterSize = (3, 3), 
                                    strides = 1, 
                                    toPadding = True, 
                                    useReLU = True, 
                                    numInputChannels = numFilters3, 
                                    maxPoolingSize=(2, 2))
dcnnArchitecture4 = CNN_Architecture(numFilters = numFilters3,
                                    filterSize = (3, 3), 
                                    strides = 1, 
                                    toPadding = True, 
                                    useReLU = True, 
                                    numInputChannels = numFilters4, 
                                    maxPoolingSize=(2, 2))

# Fully Connected 1
fc5_size = 1024 

# Lattern Code 
z_dim = 256

# flattern size
flat_size = 4096

In [4]:
class Image:
    size = 64
    channels = 3

<div>
<h1>Network Output Size</h1>
    <p>
    <ul>
        <li>Input Layer: 64 * 64 * 3 = 12288</li>
        <li>Convolutional Layer1: 32 * 32 * 32 = 32768 </li>
        <li>Convolutional Layer2: 16 * 16 * 64 = 16384 </li>
        <li>Convolutional Layer3: 8 * 8 * 128 = 8192 </li>
        <li>Convolutional Layer4: 4 * 4 * 256 = 4096 </li>
        <li>Fully Connected Layer5: 1024 </li>
        <li>Latten Code: 256 </li>
    </ul>
    </p>
</div>


In [ ]:
class DCGAN:
    """
    """
    def __init__(self, batch_size, lr_g=1e-3, lr_d=1e-4):
        self.lr_g = lr_g
        self.lr_d = lr_d
        self.batch_size = batch_size
        self.sess = tf.Session()
        
        self.build_model()
        
    def build_model(self):
        # ---------- 1. placeholder ----------
        self.lattenCode = tf.placeholder(tf.float32, shape = [None, z_dim])
        self.inputImages = tf.placeholder(tf.float32, shape = [None, Image.size, Image.size, Image.channels])
        
        # ---------- 2. Discriminator & generator ----------
        self.discriminate_true = self.build_discriminator(self.inputImages, is_training = True, 
                                                          reuse = False, init_stdev = 0.01)
        
        self.generatedImgs = self.build_generator(self.lattenCode, is_training = True, bs = self.batch_size, 
                                                  reuse = False, init_stdev = 0.01)
        
        self.dicriminate_fake = self.build_discriminator(self.generatedImgs, is_training = True, 
                                                         reuse = True, init_stdev = 0.01)
        # ---------- 3. Cost Function & Optimizer ----------
        # Discriminator loss 
        discriminator_true_loss = tf.log(self.discriminate_true)
        discriminator_fake_loss = tf.log(1 - self.dicriminate_fake)
#         discriminator_fake_loss = -tf.log(self.dicriminate_fake)
        
        discriminator_reward = tf.reduce_mean(discriminator_true_loss + discriminator_fake_loss)
        self.discriminator_loss = -1.0 * discriminator_reward
        
        # Generator Loss 
#         self.generator_loss = tf.reduce_mean(discriminator_fake_loss)
        self.generator_loss = tf.reduce_mean(-tf.log(self.dicriminate_fake))
        
        # Optimizer 
        all_vars = tf.trainable_variables()
        d_vars = [var for var in all_vars if 'discriminator' in var.name]
        g_vars = [var for var in all_vars if 'generator' in var.name]
        
        ## For Testing
#         d_names = [v.name for v in d_vars]
#         g_names = [v.name for v in g_vars]
#         print('d_names:', d_names)
#         print('g_names', g_names)
        
        self.generator_optimizer = tf.train.AdamOptimizer(self.lr_g).minimize(self.generator_loss, var_list=g_vars)
        self.discriminator_optimizer = tf.train.AdamOptimizer(self.lr_d).minimize(self.discriminator_loss, var_list=d_vars)
        
        # ---------- 4. build Testing Model ----------
        self.numGenerates = 100
        self.fake_images = self.build_generator(self.lattenCode, is_training = False, reuse = True, 
                                                bs = self.numGenerates, init_stdev = 0.01)
            
    def build_discriminator(self, inputLayer, is_training = True, reuse = False, init_stdev = 0.01):
        """
        Arg:
        -------------------------------------------------------------------------
          * inputLayer [tensor] shape = (batch, height, width, channel).
          
        """
        with tf.variable_scope("discriminator", reuse = reuse):
            # Convolutional Layers
            conv1, _ = new_convLayer(inputLayer, cnnArchitecture1, name = "dis_conv1", stdev = init_stdev)
            conv2, _ = new_convLayer(conv1, cnnArchitecture2, name = "dis_conv2", stdev = init_stdev)
            conv2 = bn(conv2, is_training = is_training, scope = "dis_bn2")
            conv3, _ = new_convLayer(conv2, cnnArchitecture3, name = "dis_conv3", stdev = init_stdev)
            conv3 = bn(conv3, is_training = is_training, scope = "dis_bn3")
            conv4, _ = new_convLayer(conv3, cnnArchitecture4, name = "dis_conv4", stdev = init_stdev)
            conv4 = bn(conv4, is_training = is_training, scope = "dis_bn4")
            
#             ## for testing
#             _, nodes1 = flattenLayer(conv1)
#             _, nodes2 = flattenLayer(conv2)
#             _, nodes3 = flattenLayer(conv3)
#             _, nodes4 = flattenLayer(conv4)
#             nodess = [nodes1, nodes2, nodes3, nodes4]
#             for i in range(1, 5): 
#                 print('#nodes in conv{}: {}'.format(i, nodess[i - 1]))
            
            # Flattern
            flat, numNodes = flattenLayer(conv4)
#             assert numNodes == flat_size, "Something wrong while flatterning ... {} != {}".format(numNodes, flat_size)
            self.flat_size = numNodes
            
            # Fully Connected Layers
            fc5 = new_fcLayer(flat, numNodes, fc5_size, useReLU=True, name="dis_fc5", stdev=init_stdev)
            fc5 = bn(fc5, is_training = is_training, scope = "dis_bn5")
            output = new_fcLayer(fc5, fc5_size, 1, useReLU=False, name="dis_out", stdev=init_stdev)
            output = tf.nn.sigmoid(output)
            
            return output
            
    
    def build_generator(self, inputLayer, is_training = True, bs = None, reuse = False, init_stdev = 0.01):
        if bs is None:
            bs = self.batch_size
            
        with tf.variable_scope("generator", reuse = reuse):
            # de fully connected layers
            fc1 = new_fcLayer(inputLayer, z_dim, fc5_size, useReLU=True, name="gen_fc1", stdev=init_stdev)
            fc1 = bn(fc1, is_training = is_training, scope = "gen_bn1")
            fc2 = new_fcLayer(fc1, fc5_size, self.flat_size, useReLU=True, name="gen_dconv2", stdev=init_stdev)
            fc2 = bn(fc2, is_training = is_training, scope = "gen_bn2")
            
            # Reshaping
            conv2 = tf.reshape(fc2, shape=[-1, 4, 4, numFilters4])
            
            # Convolutional Layers
            conv2 = unpool(conv2, kernel=(2, 2))
            conv3 = new_dconvLayer(conv2, dcnnArchitecture4, outputShape = [bs, 8, 8, 128], 
                                   name = "gen_dconv3", stdev = init_stdev)
            conv3 = bn(conv3, is_training = is_training, scope = "gen_bn3")
            
            conv3 = unpool(conv3, kernel=(2, 2))
            conv4 = new_dconvLayer(conv3, dcnnArchitecture3, outputShape = [bs, 16, 16, 64], 
                                   name = "gen_dconv4", stdev = init_stdev)
            conv4 = bn(conv4, is_training = is_training, scope = "gen_bn4")
            
            conv4 = unpool(conv4, kernel=(2, 2))
            conv5 = new_dconvLayer(conv4, dcnnArchitecture2, outputShape = [bs, 32, 32, 32], 
                                   name = "gen_dconv5", stdev = init_stdev)
            conv5 = bn(conv5, is_training = is_training, scope = "gen_bn5")
            
            conv5 = unpool(conv5, kernel=(2, 2))
            image = new_dconvLayer(conv5, dcnnArchitecture1, outputShape = [bs, Image.size, Image.size, Image.channels], 
                                   name = "gen_image", stdev = init_stdev)
            image = tf.nn.sigmoid(image)
            
            return image
            
            
    def train(self, fileDir, epochs = 100, saveModel = False, k = 1):
        generator_lossHistory = []
        discriminator_lossHistory = []
        
        print('  * Start Training ...')
        self.sess.run(tf.global_variables_initializer())
        half_batch = self.batch_size // 2
        for epoch in range(epochs):
            print('  * processing epoch #{} '.format(epoch))
            count = 0
            for batch in readImagesIn(directory=fileDir, size=(Image.size, Image.size),
                                           noiseStdev=0.0, batch_size = (self.batch_size)*k ):
                count += 1
                # ----------- Train Discriminator k times ----------
                for i in range(k):
                    batch_imgs = batch[i*self.batch_size :(i+1)*self.batch_size]
                    batch_z = np.random.normal(0, 1, [self.batch_size, z_dim])
                    
                    feed_dict_discriminator = {self.lattenCode : batch_z, self.inputImages : batch_imgs}  

                    _, d_loss = self.sess.run([self.discriminator_optimizer, self.discriminator_loss],
                                               feed_dict = feed_dict_discriminator)
                # append loss every k times => ez to plot
                discriminator_lossHistory.append(d_loss)
                
                # ---------- Train Generator 1 time ----------
                batch_z = np.random.normal(0, 1, [self.batch_size, z_dim])
                feed_dict_generator = {self.lattenCode : batch_z}
                
                _, g_loss = self.sess.run([self.generator_optimizer, self.generator_loss], 
                                          feed_dict = feed_dict_generator)
                generator_lossHistory.append(g_loss)
                
                
                # ---------- Print Loss every t times ----------
                if count % 250 == 1:
                    print('\tepoch #{} , iterations #{}, g_loss = {}, d_loss = {}'.format(epoch, count, g_loss, d_loss))
                    
            print('\tDONE!  g_loss = {}, d_loss = {}'.format(g_loss, d_loss))
                
        
        ## TODO...
        # Dst tensor is not initialized Error while saving model
        if saveModel:
            saver = tf.train.Saver()
            save_path = saver.save(self.sess, "../models/DCGAN/dcgan.ckpt")
            print("  * Model saved in path: %s" % save_path)
        # -------- Plot Learning Curve --------
        plt.figure()
        plt.title('Learning Curve')
        plt.xlabel('# Iterations')
        plt.ylabel('Cost')
        plt.plot(discriminator_lossHistory, label="Discriminator")
        plt.plot(generator_lossHistory, label="Generator")
        plt.legend(loc = 'best')
        plt.savefig('../figure/DCGAN/LearningCurve.png')
        
    def save(self, path):
        raise NotImplementedError
        
    def load(self, path):
        raise NotImplementedError
        
    def generateFakeImages(self, numImgs):
        z = np.random.normal(0, 1, size=(self.numGenerates, z_dim))
        feed_dict_test = {self.lattenCode : z }
        fakeImg = self.sess.run(self.fake_images, feed_dict=feed_dict_test)
        return fakeImg
        

In [ ]:
if __name__ == '__main__':
    directory = '../data/'
#     directory = '../faces/'
    batch_size = 128
    
    print('  * Building Model ...', end="")
    dc_gan = DCGAN(batch_size = batch_size, lr_g=1e-3, lr_d=1e-4)
    print('  Finished!!')


  * Building Model ...

In [ ]:
## Thing to do 
# 1. Add noise to input of discriminator, decay as time increasing

In [ ]:
dc_gan.train(fileDir=directory, epochs = 100, saveModel = False, k = 1)

In [ ]:
#
## ---------- Fake Images ----------
#


# z = np.random.normal(0, 5, size=(batch_size, z_dim))
# # z = np.random.uniform(-5, 5, size=(batch_size, z_dim))
# feed_dict_test = {vae.lattenCode : z }
# imgs = vae.sess.run(vae.fake_images, feed_dict=feed_dict_test)
imgs = dc_gan.generateFakeImages(dc_gan.numGenerates)
plotImages(imgs, savePath = '../figure/DCGAN/generate.png')